In [1]:
# # Modify BRAT files

# from glob import glob
# from pathlib import Path
# from shutil import copyfile
# path = "/home/ytaille/data/resources/mantra/Mantra-GSC"

# all_files = glob(path + '/*/*/*.ann')

# for file_path in all_files:
#     with open(file_path, 'r') as fp:
#         annotations = fp.readlines()
#     new_annotations = ""
#     add_comment = False
#     for i, ann in enumerate(annotations):
#         if ann[0] == "T":
#             ann_id, ann_label_b_e, ann_text = ann.split('\t')
#             new_ann_label = annotations[i+1].split(',')[-1].strip('" \n"')
#             if "Manufactured Object" not in new_ann_label:
#                 new_ann_label_b_e = " ".join([new_ann_label] + ann_label_b_e.split(' ')[1:])

#                 new_annotations += (f"{ann_id}\t{new_ann_label_b_e}\t{ann_text}")
#                 add_comment = True
#         else:
#             if add_comment:
#                 new_annotations += ann
#                 add_comment = False
            
#     new_ann_path = Path(file_path.replace('Mantra-GSC','Mantra-GSC_new_ann'))
#     new_ann_path.parent.absolute().mkdir(parents=True, exist_ok=True)œ
    
#     old_txt_path = file_path.replace('.ann', '.txt')
#     new_txt_path = old_txt_path.replace('Mantra-GSC','Mantra-GSC_new_ann')
    
    
#     copyfile(old_txt_path, new_txt_path)
#     with open(new_ann_path, 'w') as f:
#         f.write(new_annotations)


In [ ]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.insert(0,'/home/ytaille/pyner/pyner')

from pyner import NER, Vocabulary
from pyner.datasets import BRATDataset
import string
import torch
import pytorch_lightning as pl
from rich_logger import RichTableLogger

# bert_name = "bert-base-german-cased"
bert_name = "bert-base-multilingual-cased"
model = NER(
    seed=42,
    preprocessor=dict(
        module="preprocessor",
        bert_name=bert_name, # transformer name
        sentence_split_regex=r"((?:\s*\n)+\s*|(?:(?<=[a-z0-9)]\.)\s+))(?=[A-Z-])", # regex to use to split sentences (must not contain consuming patterns)
        sentence_balance_chars=('()',), # try to avoid splitting between parentheses
        sentence_entity_overlap="split", # raise when an entity spans more than one sentence, or use "split" to split entities in 2 when this happens
        word_regex='[\\w\']+|[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', # regex to use to extract words (will be aligned with bert tokens), leave to None to use wordpieces as is
        substitutions=( # Apply these regex substitutions on sentences before tokenizing
            (r"(?<=[{}\\])(?![ ])".format(string.punctuation), r" "), # insert a space before punctuations
            (r"(?<![ ])(?=[{}\\])".format(string.punctuation), r" "), # insert a space after punctuations
            #("(?<=[a-zA-Z])(?=[0-9])", r" "), # insert a space between letters and numbers
            #("(?<=[0-9])(?=[A-Za-z])", r" "), # insert a space between numbers and letters
        ),
        max_tokens=512,         # Maximum number of tokens in a sentence (will split if more than this number)
                                # Must be equal to or lower than the max number of tokens in the Bert model
        large_sentences="equal-split", # for these large sentences, split them in equal sub sentences < max_tokens tokens 
        empty_entities="raise", # when an entity cannot be mapped to any word, "raise" or "drop"
        vocabularies=torch.nn.ModuleDict({ # vocabularies to use, call .train() before initializing to fill/complete them automatically from training data
            "char": Vocabulary(string.punctuation + string.ascii_letters + string.digits, with_unk=True, with_pad=True),
            "label": Vocabulary(with_unk=True, with_pad=False),
        }).train(),
    ),

    # Word encore parameters
    word_encoders=[
        dict(
            module="char_cnn",
            n_chars=None, # automatically inferred from data
            in_channels=8,
            out_channels=50,
            kernel_sizes=(3, 4, 5),
        ),
        dict(
            module="bert",
            path=bert_name,
            n_layers=4,
            freeze_n_layers=0, # unfreeze all
            dropout_p=0.1,
        )
    ],
    
    # Decoder parameters
    decoder=dict(
        module="exhaustive_biaffine_ner",
        dim=192,
        label_dim=64,
        n_labels=None, # automatically inferred from data
        dropout_p=0.,
        use_batch_norm=False,
        contextualizer=dict(
            module="lstm",
            # use gate = False for better performance but slower convergence (needs ~50 epochs)
            gate=dict(
                module="sigmoid_gate",
                ln_mode="pre",
                init_value=0,
                proj=False,
                dim=192,
            ),
            input_size=768 + 150,
            hidden_size=192,
            num_layers=4,
            dropout_p=0.,
        )
    ),

    # Initialize last classifying layer bias with log frequencies from labels in data
    init_labels_bias=True,

    batch_size=16,
    
    # Use learning rate schedules (linearly decay with warmup)
    use_lr_schedules=True,
    warmup_rate=0.1,

    gradient_clip_val=5.,
    
    # Learning rates
    main_lr=1.5e-3,
    top_lr=1.5e-3,
    bert_lr=4e-5,
    
    # Optimizer, can be class or str
    optimizer_cls="transformers.AdamW",
).train()

flt_format = (5, "{:.4f}".format)
trainer = pl.Trainer(
    gpus=1,
    progress_bar_refresh_rate=False,
    move_metrics_to_cpu=True,
    logger=[
        #        pl.loggers.TestTubeLogger("path/to/logs", name="my_experiment"),
        RichTableLogger(key="epoch", fields={
            "epoch": {},
            "step": {},
            "train_loss": {"goal": "lower_is_better", "format": "{:.4f}"},
            "train_f1": {"goal": "higher_is_better", "format": "{:.4f}", "name": "train_f1"},
            "train_precision": {"goal": "higher_is_better", "format": "{:.4f}", "name": "train_p"},
            "train_recall": {"goal": "higher_is_better", "format": "{:.4f}", "name": "train_r"},

            "val_loss": {"goal": "lower_is_better", "format": "{:.4f}"},
            "val_f1": {"goal": "higher_is_better", "format": "{:.4f}", "name": "val_f1"},
            "val_precision": {"goal": "higher_is_better", "format": "{:.4f}", "name": "val_p"},
            "val_recall": {"goal": "higher_is_better", "format": "{:.4f}", "name": "val_r"},

            "main_lr": {"format": "{:.2e}"},
            "top_lr": {"format": "{:.2e}"},
            "bert_lr": {"format": "{:.2e}"},
        }
                       
       ),
    ],
    max_epochs=50)

dataset = BRATDataset(
    train=[
        "/home/ytaille/data/resources/conll/brat/eng/train",
        "/home/ytaille/data/resources/conll/brat/ger/train",
    ],
#     test="/home/ytaille/data/resources/quaero/corpus/test/MEDLINE",#"path/to/brat/test",    # None for training only, test directory otherwise
    val="/home/ytaille/data/resources/conll/brat/ger/test",  # first 20% doc will be for validation
    seed=42,  # don't shuffle before splitting4
    ger_num=1000,
)

trainer.fit(model, dataset)
model.save_pretrained("conll_ner.pt")


GPU available: True, used: True
TPU available: None, using: 0 TPU cores


INIT RICH TABLE LOGGER YT_NLP
NUMBER OF ENGLISH FILES: 7374
NUMBER OF GERMAN FILES: 1000


Token indices sequence length is longer than the specified maximum sequence length for this model (520 > 512). Running this sequence through the model will result in indexing errors
/home/ytaille/pyner/pyner/modules.py:384: UserWarning: Sentences > {self.max_tokens} tokens will be split with option large_sentence="{self.large_sentences}". Consider using a more restrictive regex for sentence splitting if you want to avoid it.
  warnings.warn('Sentences > {self.max_tokens} tokens will be split with option large_sentence="{self.large_sentences}". Consider using a more restrictive regex for sentence splitting if you want to avoid it.')
/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/torch/tensor.py:904: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:930.)
  return super(Tensor, sel

/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-6d47879e9e4c>", line 145, in <module>
    trainer.fit(model, dataset)
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 498, in fit
    self.dispatch()
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 545, in dispatch
    self.accelerator.start_training(self)
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/pytorch_lightning/accelerators/accelerator.py", line 73, in start_training
    self.training_type_plugin.start_training(trainer)
  File "/home/ytaille/.conda/envs/yt_nlp/lib/python3.7/site-packages/pytorch_lightning/plugins/training_type/training_type_plugin.py", line 114, in start_training
    self._r

TypeError: object of type 'NoneType' has no len()

In [ ]:
# f1: 56 max vs 82+ avec english contre german
# 50/50: 77 -> essayer avec seulement ger
# 90 eng / 10 ger -> 75.8. 72 avec 740 docs ger
# 100 ger docs only: 0.48
# 100 ger docs + full eng: 0.67
# 300 ger docs + full eng: 0.73
# full eng + full ger: 0.85 (7.3k + 23k)

# OFFICIAL CONLL 2003: 
# ENG: train: 946, dev: 216, test: 231
# GER: train: 553, dev: 201, test: 155
# 9736 sentences in train -> should be 12,705, but number of docs matches -> wrong sentence chunking but may not impact results that much

# IMPORTANT: I is for B/I, B is only for new entities of same type


# GERMAN TO GERMAN:
# 85+ WITH PYNER

# ger: train: 23951 (3.7 MB), dev: 2200, test: 5080 sentences
# eng: train: 7374 (1.7MB), dev: 1874, test: 1626 sentences

# tester avec genre 10k sentences , 90/10, 70/30 50/50 etc

# CHECK FOR INFORMATION LEAKS IN GERMAN CORPUS
# Test Microsoft model
# Test without pyner

In [ ]:
# train on quaero and test on Mantra
# deft, quaero

# compte rendus anglais étiquetés en concepts style UMLd (procédure, ANAT), puis utiliser corpus francais

# Look for multilingual corpuses 

# See tradeof between proportion of english corpus and target language -> at what proportion of target language are the results comparable with full target language.
